In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))

In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from cycler import cycler

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.autograd import Variable
import torch.nn.functional as F

import torchvision.transforms as transforms
import torchvision.utils as vutils
from torchvision.utils import save_image
from torchvision.transforms import ToPILImage, Resize
import torchvision.datasets as dset
to_img  = ToPILImage()
resize_img  = Resize(200)
def show_img(img):
    return resize_img(to_img(img))

cmap=plt.cm.tab10
c = cycler('color', cmap(np.linspace(0,1,10)))
plt.rcParams["axes.prop_cycle"] = c

%config InlineBackend.figure_format = 'retina'
%matplotlib notebook
%matplotlib notebook

%load_ext autoreload
%autoreload 2

def img_ae(x):
    x = 0.5 * (x + 1)
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 1, 64, 64)
    return x

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
%matplotlib notebook
%matplotlib notebook

In [19]:
dataset = dset.ImageFolder(root='/scratch/rag394/data/gaussian_generator/',
                           transform=transforms.Compose([
                                   transforms.Grayscale(),
                                   transforms.Resize(64),
                                   transforms.CenterCrop(64),
                                   transforms.ToTensor(),
                                   transforms.Normalize((0.5,),(0.5,)),
                               ]))

dataloader = torch.utils.data.DataLoader(dataset, batch_size=1,
                                         shuffle=False, num_workers=32)

targets = pd.read_csv('/scratch/rag394/data/gaussian_generator/gaussian_parameters.csv').drop(['Unnamed: 0'], 1).values
targets = Variable(torch.from_numpy(targets).type(torch.FloatTensor), requires_grad=False)
targets = targets.cuda()

In [63]:
class GaussianModel(nn.Module):
    def __init__(self):
        super(GaussianModel, self).__init__()
        
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 16, 3, stride=3, padding=1),  # b, 16, 10, 10
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=2),  # b, 16, 5, 5
            nn.Conv2d(16, 8, 3, stride=2, padding=1),  # b, 8, 3, 3
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=1), # b, 8, 2, 2
            )
        
        self.projector = nn.Linear(200,4)

    def forward(self, x):
        x = self.encoder(x)
        return self.projector(x.view(-1))

In [67]:
model(img).data


 0.6165
 0.0483
 0.0502
 0.7126
[torch.cuda.FloatTensor of size 4 (GPU 0)]

In [53]:
model = GaussianModel()
model.cuda()
optimizer = torch.optim.Adam(model.parameters(),lr=1.0e-3)
criterion = nn.MSELoss()
total_loss = []

In [75]:
[x for x in [y.cpu().numpy() for y in outputs]]

[array([0.6145753 , 0.04292139, 0.04940778, 0.7361835 ], dtype=float32),
 array([0.60986125, 0.03201647, 0.03270033, 0.7521088 ], dtype=float32),
 array([0.6122662 , 0.02064007, 0.01141953, 0.7733962 ], dtype=float32),
 array([0.618898  , 0.02400056, 0.01783472, 0.7995205 ], dtype=float32),
 array([0.62459207, 0.00502962, 0.01857679, 0.80859315], dtype=float32),
 array([ 0.6165274 , -0.0034821 ,  0.01420571,  0.7835119 ], dtype=float32),
 array([0.6271835 , 0.00384045, 0.01514437, 0.8461282 ], dtype=float32),
 array([ 0.6227471 , -0.00666573, -0.01193001,  0.80587953], dtype=float32),
 array([ 0.6296824 , -0.00472699, -0.02304538,  0.80447197], dtype=float32),
 array([ 0.6445835 , -0.00118558,  0.0125591 ,  0.83925116], dtype=float32),
 array([0.65898883, 0.00120236, 0.00811237, 0.82592696], dtype=float32),
 array([ 6.6789401e-01,  1.6323373e-02, -3.9048120e-04,  8.3820134e-01],
       dtype=float32),
 array([ 6.6859108e-01, -2.3158733e-04,  1.0055102e-02,  8.9314508e-01],
       dtype

In [68]:
num_epochs = 5
outputs = []
for epoch in range(num_epochs):
    for k, data in enumerate(dataloader):
        img, _ = data
        img = Variable(img).cuda()
        # ===================forward=====================
        output = model(img)
        outputs.append(output.data)
        loss   = criterion(output, targets[k,:])
        
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}'
          .format(epoch+1, num_epochs, loss.data[0]))
    total_loss.append(loss.data[0])

torch.save(model.state_dict(), '/scratch/rag394/data/ae/conv_gaussian_encoder.pth')

Process Process-470:
Process Process-450:
Process Process-460:
Process Process-459:
Process Process-473:
Process Process-462:
Process Process-471:
Process Process-451:
Process Process-476:
Process Process-469:
Process Process-467:
Process Process-480:
Process Process-479:
Process Process-474:
Process Process-466:
Process Process-458:
Process Process-454:
Process Process-453:
Process Process-477:
Process Process-463:
Process Process-478:
Process Process-452:
Process Process-475:
Process Process-472:
Process Process-464:
Process Process-455:
Process Process-449:
Process Process-461:
Process Process-456:
Process Process-457:
Process Process-468:
Process Process-465:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (mo

  File "/share/apps/python3/3.6.3/intel/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/share/apps/python3/3.6.3/intel/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/share/apps/python3/3.6.3/intel/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/share/apps/python3/3.6.3/intel/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/share/apps/python3/3.6.3/intel/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/share/apps/python3/3.6.3/intel/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/share/apps/python3/3.6.3/intel/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, *

  File "/share/apps/python3/3.6.3/intel/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/share/apps/python3/3.6.3/intel/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/share/apps/python3/3.6.3/intel/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/share/apps/python3/3.6.3/intel/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/share/apps/python3/3.6.3/intel/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/share/apps/python3/3.6.3/intel/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/share/apps/python3/3.6.3/intel/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/share/apps/python3/3.6.3/intel/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/share/apps/python3/3.6.

KeyboardInterrupt: 

  File "/share/apps/python3/3.6.3/intel/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/share/apps/python3/3.6.3/intel/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/share/apps/python3/3.6.3/intel/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/share/apps/python3/3.6.3/intel/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/share/apps/python3/3.6.3/intel/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/share/apps/python3/3.6.3/intel/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/share/apps/python3/3.6.3/intel/lib/python3.6/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self

In [ ]:
fig, ax = plt.subplots(figsize=(9,7))
pd.Series(total_loss, name='MSE Loss').plot(ax=ax,color='darkred', 
                                            lw=0.6, marker='s', markersize=3., legend=True)
ax.grid(alpha=0.2)
ax.set_xlabel('Epoch')

ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_alpha(False)
ax.grid(alpha=0.3)

### Notes 

In [ ]:
class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 16, 3, stride=3, padding=1),  # b, 16, 10, 10
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=2),  # b, 16, 5, 5
            nn.Conv2d(16, 8, 3, stride=2, padding=1),  # b, 8, 3, 3
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=1), # b, 8, 2, 2
            nn.Linear(200,4))
        
        self.decoder = nn.Sequential(
            nn.Linear(4,200),
            nn.ConvTranspose2d(8, 16, 3, stride=2),  # b, 16, 5, 5
            nn.ReLU(True),
            nn.ConvTranspose2d(16, 8, 5, stride=3, padding=1),  # b, 8, 15, 15
            nn.ReLU(True),
            nn.ConvTranspose2d(8, 1, 2, stride=2, padding=1),  # b, 1, 28, 28
            nn.Tanh())

    def forward(self, x):
        enc = self.encoder(x)
        dec = self.decoder(enc)
        
        return enc, dec

In [ ]:
class GaussianModel(nn.Module):
    def __init__(self):
        super(GaussianModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, stride=3, padding=1)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 4)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x